## Análise Fatorial e PCA
### MBA em Data Science e Analytics USP ESALQ

**Prof Dr.** Wilson Tarantin Junior

**Aluna:** Luiza Batista Laquini

**Turma:** DSA 2024-1

### Bibliotecas e configurações

In [ ]:
import pandas as pd
import numpy as np
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import sympy as sy
import scipy as sp

#import plotly.io as pio
#pio.renderers.default='browser'

import warnings
warnings.filterwarnings("ignore")

### Visualização dos dados

In [ ]:
notas = pd.read_excel("notas_fatorial.xlsx")
# Fonte: FÃ¡vero e Belfiore (2024, CapÃ­tulo 10)

% InformaÃ§Ãµes sobre as variÃ¡veis

InformaÃ§Ãµes gerais sobre o DataFrame

In [ ]:
print(notas.info())

EstatÃ­sticas descritiva das variÃ¡veis

In [ ]:
print(notas.describe())

% Separando somente as variÃ¡veis quantitativas do banco de dados

In [ ]:
notas_pca = notas[["finanÃ§as", "custos", "marketing", "atuÃ¡ria"]]

% Matriz de correlaÃ§Ãµes de Pearson entre as variÃ¡veis

In [ ]:
pg.rcorr(notas_pca, method = 'pearson', upper = 'pval', 
         decimals = 4, 
         pval_stars = {0.01: '***', 0.05: '**', 0.10: '*'})

% Outra maneira de analisar as informaÃ§Ãµes das correlaÃ§Ãµes

Matriz de correlaÃ§Ãµes em um objeto "simples"

In [ ]:
corr = notas_pca.corr()

GrÃ¡fico interativo

In [ ]:
fig = go.Figure()

In [ ]:
fig.add_trace(
    go.Heatmap(
        x = corr.columns,
        y = corr.index,
        z = np.array(corr),
        text=corr.values,
        texttemplate='%{text:.4f}',
        colorscale='viridis'))

In [ ]:
fig.update_layout(
    height = 600,
    width = 600,
    yaxis=dict(autorange="reversed"))

In [ ]:
fig.show()

% Teste de Esfericidade de Bartlett

In [ ]:
bartlett, p_value = calculate_bartlett_sphericity(notas_pca)

In [ ]:
print(f'QuiÂ² Bartlett: {round(bartlett, 2)}')
print(f'p-valor: {round(p_value, 4)}')

% Definindo a PCA (procedimento inicial com todos os fatores possÃ­veis)

In [ ]:
fa = FactorAnalyzer(n_factors=4, method='principal', rotation=None).fit(notas_pca)

% Obtendo os eigenvalues (autovalores): resultantes da funÃ§Ã£o FactorAnalyzer

In [ ]:
autovalores = fa.get_eigenvalues()[0]

In [ ]:
print(autovalores) # Temos 4 autovalores, pois sÃ£o 4 variÃ¡veis ao todo

Soma dos autovalores

In [ ]:
round(autovalores.sum(), 2)

% Obtendo os autovalores e autovetores: ilustrando o fundamento

 AtenÃ§Ã£o: esta cÃ©lula tem fins didÃ¡ticos, nÃ£o Ã© requerida na FactorAnalyzer

Parametrizando o pacote

In [ ]:
lamda = sy.symbols('lamda')
sy.init_printing(scale=0.8)

Especificando a matriz de correlaÃ§Ãµes

In [ ]:
matriz = sy.Matrix(corr)
polinomio = matriz.charpoly(lamda)

In [ ]:
polinomio

Obtendo as raÃ­zes do polinÃ´mio caracterÃ­stico: sÃ£o os autovalores

In [ ]:
autovalores, autovetores = sp.linalg.eigh(corr)
autovalores = autovalores[::-1]

Obtendo os autovetores para cada autovalor extraÃ­do

In [ ]:
autovetores = autovetores[:, ::-1]

% Eigenvalues, variÃ¢ncias e variÃ¢ncias acumuladas

In [ ]:
autovalores_fatores = fa.get_factor_variance()

In [ ]:
tabela_eigen = pd.DataFrame(autovalores_fatores)
tabela_eigen.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_eigen.columns)]
tabela_eigen.index = ['Autovalor','VariÃ¢ncia', 'VariÃ¢ncia Acumulada']
tabela_eigen = tabela_eigen.T

In [ ]:
print(tabela_eigen)

% GrÃ¡fico da variÃ¢ncia acumulada dos componentes principais

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.barplot(x=tabela_eigen.index, y=tabela_eigen['VariÃ¢ncia'], data=tabela_eigen, palette='rocket')
ax.bar_label(ax.containers[0])
plt.title("Fatores ExtraÃ­dos", fontsize=16)
plt.xlabel(f"{tabela_eigen.shape[0]} fatores que explicam {round(tabela_eigen['VariÃ¢ncia'].sum()*100,2)}% da variÃ¢ncia", fontsize=12)
plt.ylabel("Porcentagem de variÃ¢ncia explicada", fontsize=12)
plt.show()

% Determinando as cargas fatoriais

In [ ]:
cargas_fatoriais = fa.loadings_

In [ ]:
tabela_cargas = pd.DataFrame(cargas_fatoriais)
tabela_cargas.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_cargas.columns)]
tabela_cargas.index = notas_pca.columns

In [ ]:
print(tabela_cargas)

% GrÃ¡fico das cargas fatoriais (loading plot)

In [ ]:
plt.figure(figsize=(12,8))
tabela_cargas_chart = tabela_cargas.reset_index()
plt.scatter(tabela_cargas_chart['Fator 1'], tabela_cargas_chart['Fator 2'], s=50, color='red')

In [ ]:
def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x'] + 0.05, point['y'], point['val'])

In [ ]:
label_point(x = tabela_cargas_chart['Fator 1'],
            y = tabela_cargas_chart['Fator 2'],
            val = tabela_cargas_chart['index'],
            ax = plt.gca()) 

In [ ]:
plt.axhline(y=0, color='grey', ls='--')
plt.axvline(x=0, color='grey', ls='--')
plt.ylim([-1.1,1.1])
plt.xlim([-1.1,1.1])
plt.title("Loading Plot", fontsize=16)
plt.xlabel(f"Fator 1: {round(tabela_eigen.iloc[0]['VariÃ¢ncia']*100,2)}% de variÃ¢ncia explicada", fontsize=12)
plt.ylabel(f"Fator 2: {round(tabela_eigen.iloc[1]['VariÃ¢ncia']*100,2)}% de variÃ¢ncia explicada", fontsize=12)
plt.show()

% Determinando as comunalidades

In [ ]:
comunalidades = fa.get_communalities()

In [ ]:
tabela_comunalidades = pd.DataFrame(comunalidades)
tabela_comunalidades.columns = ['Comunalidades']
tabela_comunalidades.index = notas_pca.columns

In [ ]:
print(tabela_comunalidades)

% ExtraÃ§Ã£o dos fatores para as observaÃ§Ãµes do banco de dados

In [ ]:
fatores = pd.DataFrame(fa.transform(notas_pca))
fatores.columns =  [f"Fator {i+1}" for i, v in enumerate(fatores.columns)]

Adicionando os fatores ao banco de dados

In [ ]:
notas = pd.concat([notas.reset_index(drop=True), fatores], axis=1)

% Identificando os scores fatoriais

In [ ]:
scores = fa.weights_

In [ ]:
tabela_scores = pd.DataFrame(scores)
tabela_scores.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_scores.columns)]
tabela_scores.index = notas_pca.columns

In [ ]:
print(tabela_scores)

% CorrelaÃ§Ã£o entre os fatores extraÃ­dos

A seguir, verifica-se que a correlaÃ§Ã£o entre os fatores Ã© zero (ortogonais)

In [ ]:
pg.rcorr(notas[['Fator 1','Fator 2', 'Fator 3', 'Fator 4']],
         method = 'pearson', upper = 'pval', 
         decimals = 4, 
         pval_stars = {0.01: '***', 0.05: '**', 0.10: '*'})

% CritÃ©rio de Kaiser (raiz latente)

Verificar os autovalores com valores maiores que 1<br>
Existem dois componentes maiores do que 1

% Parametrizando a PCA para dois fatores (autovalores > 1)

In [ ]:
fa = FactorAnalyzer(n_factors=2, method='principal', rotation=None).fit(notas_pca)

% Eigenvalues, variÃ¢ncias e variÃ¢ncias acumuladas de 2 fatores

Note que nÃ£o hÃ¡ alteraÃ§Ãµes nos valores, apenas ocorre a seleÃ§Ã£o dos fatores

In [ ]:
autovalores_fatores = fa.get_factor_variance()

In [ ]:
tabela_eigen = pd.DataFrame(autovalores_fatores)
tabela_eigen.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_eigen.columns)]
tabela_eigen.index = ['Autovalor','VariÃ¢ncia', 'VariÃ¢ncia Acumulada']
tabela_eigen = tabela_eigen.T

In [ ]:
print(tabela_eigen)

% Determinando as cargas fatoriais

Note que nÃ£o hÃ¡ alteraÃ§Ãµes nas cargas fatoriais nos 2 fatores!

In [ ]:
cargas_fatoriais = fa.loadings_

In [ ]:
tabela_cargas = pd.DataFrame(cargas_fatoriais)
tabela_cargas.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_cargas.columns)]
tabela_cargas.index = notas_pca.columns

In [ ]:
print(tabela_cargas)

% Determinando as novas comunalidades

As comunalidades sÃ£o alteradas, pois hÃ¡ fatores retirados da anÃ¡lise!

In [ ]:
comunalidades = fa.get_communalities()

In [ ]:
tabela_comunalidades = pd.DataFrame(comunalidades)
tabela_comunalidades.columns = ['Comunalidades']
tabela_comunalidades.index = notas_pca.columns

In [ ]:
print(tabela_comunalidades)

% ExtraÃ§Ã£o dos fatores para as observaÃ§Ãµes do banco de dados

Vamos remover os fatores obtidos anteriormente

In [ ]:
notas = notas.drop(columns=['Fator 1', 'Fator 2', 'Fator 3', 'Fator 4'])

 Vamos gerar novamente, agora para os 2 fatores extraÃ­dos

In [ ]:
fatores = pd.DataFrame(fa.transform(notas_pca))
fatores.columns =  [f"Fator {i+1}" for i, v in enumerate(fatores.columns)]

Adicionando os fatores ao banco de dados

In [ ]:
notas = pd.concat([notas.reset_index(drop=True), fatores], axis=1)

Note que sÃ£o os mesmos, apenas ocorre a seleÃ§Ã£o dos 2 primeiros fatores!

% Identificando os scores fatoriais

NÃ£o hÃ¡ mudanÃ§as nos scores fatoriais!

In [ ]:
scores = fa.weights_

In [ ]:
tabela_scores = pd.DataFrame(scores)
tabela_scores.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_scores.columns)]
tabela_scores.index = notas_pca.columns

In [ ]:
print(tabela_scores)

% Criando um ranking (soma ponderada e ordenamento)

O ranking irÃ¡ considerar apenas os 2 fatores com autovalores > 1<br>
A base de seleÃ§Ã£o Ã© a tabela_eigen

In [ ]:
notas['Ranking'] = 0

In [ ]:
for index, item in enumerate(list(tabela_eigen.index)):
    variancia = tabela_eigen.loc[item]['VariÃ¢ncia']
    notas['Ranking'] = notas['Ranking'] + notas[tabela_eigen.index[index]]*variancia
    
print(notas)

% Fim!